# MUSO Groups Close_OUT Reports

---

Per groups:

1. date graduation is valid and in the reporting.
2. number of member.
3. number of HC left.
4. number of HC_2022.
5. number of indirect beneficiaires.

## 

## Preliminaries

In [1]:
from pandas import read_sql_query, read_excel
import numpy as np
from datetime import datetime
from dotenv import load_dotenv
from decouple import config
from sqlalchemy import create_engine, text
from dateutil.parser import parse
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from IPython.core.interactiveshell import InteractiveShell





InteractiveShell.ast_node_interactivity = 'all'

load_dotenv()
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')



In [2]:
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

In [3]:
# Non-applicable 
MHC22 = f"""
SELECT 
    b.caseid, mhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (!ISNULL(indice_householdcount)
            OR (is_household_applicable = 'yes'))) mhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mhc22.patient_code = b.patient_code
"""



# applicable 
NOMHC22 = f"""
SELECT 
    b.caseid, nomhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (ISNULL(indice_householdcount)
            AND (is_household_applicable = 'no'
            OR ISNULL(is_household_applicable)))) nomhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON nomhc22.patient_code = b.patient_code
"""

MUSO_HOUSEHOLD_22 = f"""
SELECT 
    b.caseid, mh22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.muso_household_2022) mh22
        LEFT JOIN
    (SELECT 
        id, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mh22.id_patient = b.id
"""

In [4]:
mhc22 = read_sql_query(text(MHC22),engine.connect(),parse_dates=True)
nomhc22 = read_sql_query(text(NOMHC22),engine.connect(),parse_dates=True)
mh22 = read_sql_query(text(MUSO_HOUSEHOLD_22),engine.connect(),parse_dates=True)

In [5]:
engine.dispose()

In [6]:
#load_dotenv('./id_cc.env')
email = config('COMCARE_EMAIL')
password_cc = config('COMCARE_PASSWORD')

driver = webdriver.Chrome()
driver.implicitly_wait(1000)

#Creating login function
def muso_beneficiaries():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/91fe1e2fc809dfd56a29bfce97cc9d15/'
    )
    #driver.find_element_by_xpath('//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element(By.XPATH,'//*[@id="id_auth-username"]').send_keys(email)
    #driver.find_element_by_xpath('//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.XPATH,'//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.CSS_SELECTOR,'button[type=submit]').click()

#Muso beneficiaries
muso_beneficiaries()

#Download the database "All gardens"
#driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,"//*[@id='download-export-form']/form/div[2]/div/div[2]/div[1]/button/span[1]").click()
#driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()    
driver.find_element(By.XPATH,"//*[@id='download-progress']/div/div/div[2]/div[1]/form/a/span[1]").click()    

#-----------------------------------------------------------------------------------------------------------------------

#Household count
def muso_groups():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/91fe1e2fc809dfd56a29bfce973b4c0d/'
    )
    
muso_groups()

#Download the database "Household count"
#driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
#driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

In [7]:
muso_benef = read_excel(f"~/Downloads/muso_beneficiaries {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Cases')
muso_group = read_excel(f"~/Downloads/muso_groupes {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")

- groups actifs
- benef actifs
- beneficiaires dans groupes actifs

In [8]:
muso_group.columns

Index(['number', 'caseid', 'name', 'gps', 'gps_date', 'cycle_1_start_date',
       'is_graduated', 'office', 'graduation_date', 'localite_name',
       'cycle_2_start_date', 'commune', 'officer', 'cycle_3_end_date',
       'commune_name', 'code', 'cycle_2_end_date', 'cycle_1_end_date',
       'inactive_date', 'creation_date', 'meeting_hour', 'officer_name',
       'meeting_day', 'formed_by_members_from', 'localite', 'actual_cycle',
       'cycle_3_start_date', 'is_inactive', 'closed', 'closed_by_username',
       'closed_date', 'last_modified_by_user_username', 'last_modified_date',
       'opened_by_username', 'opened_date', 'owner_name', 'case_link',
       'case_type', 'closed_by_user_id', 'external_id',
       'last_modified_by_user_id', 'opened_by_user_id', 'owner_id',
       'server_last_modified_date', 'state', 'office_name', 'owner_id1',
       'section', 'section_name', 'departement', 'departement_name', 'adress',
       'office_location_id', 'commune_location_id',
       'sec

In [11]:
muso_benef.columns

Index(['number', 'caseid', 'name', 'dob', 'first_name', 'household_number',
       'last_name', 'patient_code', 'group_code', 'group_commune', 'phone',
       'is_inactive', 'group_departement', 'inactive_date', 'graduated',
       'abandoned_date', 'is_abandoned', 'graduation_date', 'gender', 'rank',
       'group_name', 'address', 'closed', 'closed_by_username', 'closed_date',
       'last_modified_by_user_username', 'last_modified_date',
       'opened_by_username', 'opened_date', 'owner_name', 'case_link',
       'case_type', 'closed_by_user_id', 'external_id',
       'last_modified_by_user_id', 'opened_by_user_id', 'owner_id',
       'server_last_modified_date', 'state', 'loan_amount', 'owner_id1',
       'is_caris_member', 'is_pvvih', 'last_loan_date', 'officer',
       'meeting_hour', 'code', 'is_graduated', 'formed_by_members_from',
       'actual_cycle', 'commune', 'departement', 'officer_name',
       'localite_name', 'departement_name', 'creation_date', 'section',
       'lo

In [9]:
driver.quit()
